# Data Loading

In [1]:
from sklearn.datasets import  fetch_california_housing
housing = fetch_california_housing()

In [2]:
for i in housing:
  print(i)

data
target
frame
target_names
feature_names
DESCR


# Data preprocessing

In [3]:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# Model Building

In [1]:
import tensorflow as tf
from tensorflow import keras

![Wide and Deep Model](custom_model.png)

In [5]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs): # **kwargs needed for naming the model. Units is the number of neurons in the hidden layer. 
        super().__init__(**kwargs)  # needed to support naming the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)    
    def call(self, inputs): # inputs is a tuple containing the wide and deep inputs
        input_wide, input_deep = inputs # unpack the tuple
        norm_wide = self.norm_layer_wide(input_wide) # normalize the wide input
        norm_deep = self.norm_layer_deep(input_deep) # normalize the deep input
        hidden1 = self.hidden1(norm_deep) # pass the normalized deep input through the first hidden layer
        hidden2 = self.hidden2(hidden1) # pass the output of the first hidden layer through the second hidden layer
        concat = tf.keras.layers.concatenate([norm_wide, hidden2]) # concatenate the wide input and the output of the second hidden layer. Mixing wide and deep input. 
        output = self.main_output(concat) # pass the concatenated input through the output layer.
        return output

In [6]:
tf.random.set_seed(42)
model = WideAndDeepModel(30, activation="relu", name="my_model") # create an instance of the model. 30 is the number of neurons in the hidden layer. input shape is inferred from the input data.

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, 
              metrics=["RootMeanSquaredError"])

In [10]:
# We have 7 features. The first 5 are wide features and the last 5 are deep features.
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

# Model Training

In [11]:
model.norm_layer_wide.adapt(X_train_wide) # normalize the wide input. Adapt is used to calculate the mean and standard deviation of the train data. We use this to normalize the validation and test data.
model.norm_layer_deep.adapt(X_train_deep) # normalize the deep input. 

history = model.fit((X_train_wide, X_train_deep), # pass the wide and deep input as a tuple 
                    y_train, # target
                    validation_data=((X_valid_wide, X_valid_deep), y_valid), 
                    epochs=10) 

Epoch 1/10
363/363 [==============================] - 3s 4ms/step - loss: 2.0211 - root_mean_squared_error: 1.4217 - val_loss: 2.0318 - val_root_mean_squared_error: 1.4254
Epoch 2/10
363/363 [==============================] - 1s 4ms/step - loss: 0.6697 - root_mean_squared_error: 0.8183 - val_loss: 3.4732 - val_root_mean_squared_error: 1.8637
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.5667 - root_mean_squared_error: 0.7528 - val_loss: 1.2420 - val_root_mean_squared_error: 1.1145
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4724 - root_mean_squared_error: 0.6873 - val_loss: 1.1187 - val_root_mean_squared_error: 1.0577
Epoch 5/10
363/363 [==============================] - 1s 4ms/step - loss: 0.4237 - root_mean_squared_error: 0.6510 - val_loss: 2.4028 - val_root_mean_squared_error: 1.5501
Epoch 6/10
363/363 [==============================] - 1s 3ms/step - loss: 0.4154 - root_mean_squared_error: 0.6445 - val_loss: 2.0498 - val_

# Model Evaluation

Model evaluation is done using the evaluate method. The evaluate method returns the loss and the metrics specified in the compile method. Evaluate method takes the input and target as arguments.

In [12]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
eval_results

162/162 [==============================] - 0s 2ms/step - loss: 0.3589 - root_mean_squared_error: 0.5991


[0.35890090465545654, 0.5990833640098572]

# Prediction

In [13]:
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3] # get the first 3 instances of the wide and deep input

In [14]:
y_pred = model.predict((X_new_wide, X_new_deep))
y_pred

1/1 [==============================] - 0s 113ms/step


array([[0.4832632],
       [1.6469457],
       [3.5325484]], dtype=float32)

In [15]:
y_test[:3]

array([0.477  , 0.458  , 5.00001])